<a href="https://colab.research.google.com/github/LEECHANGDAEE/colab/blob/main/xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import xgboost
train = pd.read_csv('./train1.csv',encoding='cp949')
test = pd.read_csv('./test1.csv',encoding='cp949')
submission = pd.read_csv('./sample_submission1.csv',encoding='cp949')


train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i


train['지역'] = train['지역'].map(local_map)
test['지역'] = test['지역'].map(local_map)

train['전용면적'] = round(train['전용면적'])//5*5
test['전용면적'] = round(test['전용면적'])//5*5

train_apt = train[train['임대건물구분']=='아파트']
train_st = train[train['임대건물구분']=='상가']
test_apt = test[test['임대건물구분']=='아파트']
test_st = test[test['임대건물구분']=='상가']

columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스',]
target = '등록차량수'
area_columns = []

for area in train_apt['전용면적'].unique():
    area_columns.append(f'면적_{area}')

new_train = pd.DataFrame()
new_test = pd.DataFrame()

for i, code in tqdm(enumerate(train_apt['단지코드'].unique())):
    temp = train_apt[train_apt['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test_apt['단지코드'].unique())):
    temp = test_apt[test_apt['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

423it [00:09, 46.41it/s]
150it [00:03, 47.03it/s]


In [ ]:
for i in range(50) :
  
  x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2,random_state =i)
  mlr = LinearRegression()
  mlr.fit(x_train, y_train)

  ridge = Ridge(alpha = 0.5).fit(x_train, y_train)

  lasso = Lasso(alpha=0.2).fit(x_train, y_train)

  y_linear_predict = mlr.predict(x_test)
  y_ridge_predict = ridge.predict(x_test)
  y_lasso_predict = lasso.predict(x_test)
  from sklearn.metrics import mean_absolute_error
  from sklearn.metrics import mean_squared_error
  print(mean_absolute_error(y_lasso_predict,y_test))

NameError: ignored

In [ ]:

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2,random_state =0)
mlr = LinearRegression()
mlr.fit(x_train, y_train)

ridge = Ridge(alpha = 0.5).fit(x_train, y_train)

lasso = Lasso(alpha=0.2).fit(x_train, y_train)

y_linear_predict = mlr.predict(x_test)
y_ridge_predict = ridge.predict(x_test)
y_lasso_predict = lasso.predict(x_test)
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print('linear regression rmse : {}'.format(mean_squared_error(y_linear_predict,y_test)**0.5))
print('ridge regression rmse : {}'.format(mean_squared_error(y_ridge_predict,y_test)**0.5))
print('lasso regression rmse : {}'.format(mean_squared_error(y_lasso_predict,y_test)**0.5))
print(mean_absolute_error(y_lasso_predict,y_test))

In [ ]:
x_test.head()

,총세대수,공가수,단지내주차면수,지하철,버스,면적_40.0,면적_50.0,면적_60.0,면적_30.0,면적_45.0,면적_35.0,면적_25.0,면적_55.0,면적_75.0,면적_85.0,면적_20.0,면적_70.0,면적_80.0,면적_65.0,면적_15.0,지역_0.0,지역_1.0,지역_2.0,지역_3.0,지역_4.0,지역_5.0,지역_6.0,지역_7.0,지역_8.0,지역_9.0,지역_10.0,지역_11.0,지역_12.0,지역_13.0,지역_14.0,지역_15.0
0,754.0,14.0,683.0,0.0,2.0,116.0,376.0,0.0,0.0,262.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1354.0,9.0,1216.0,0.0,3.0,420.0,578.0,0.0,0.0,356.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,619.0,18.0,547.0,0.0,16.0,165.0,132.0,44.0,82.0,196.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,593.0,7.0,543.0,0.0,3.0,335.0,84.0,0.0,0.0,174.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1297.0,11.0,1112.0,0.0,2.0,425.0,304.0,0.0,0.0,568.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
x_test_scale

array([[ 0.17559366, -0.10794088,  0.38644442, ..., -0.20412415,
         0.        , -0.14285714],
       [ 1.71663993, -0.56020716,  2.0460153 , ..., -0.20412415,
         0.        , -0.14285714],
       [-0.17114175,  0.25387214, -0.03701082, ..., -0.20412415,
         0.        , -0.14285714],
       ...,
       [-1.45277857, -0.56020716, -1.61562702, ..., -0.20412415,
         0.        , -0.14285714],
       [-0.02731076,  2.06293724, -0.28610213, ..., -0.20412415,
         0.        , -0.14285714],
       [-0.77985503,  2.69611003, -0.80608025, ..., -0.20412415,
         0.        , -0.14285714]])

In [ ]:
params = {"learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
xgb_model = xgboost.XGBRegressor(objective='reg:squarederror')
xgb_model_random = RandomizedSearchCV(estimator = xgb_model, param_distributions = params, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
xgb_model_random.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
             importance_type='gain', learning_rate=0.2, max_delta_step=0,
             max_depth=15, min_child_weight=7, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [ ]:
xgb_model_random.best_params_

{'colsample_bytree': 0.7,
 'gamma': 0.2,
 'learning_rate': 0.2,
 'max_depth': 15,
 'min_child_weight': 7}

In [ ]:
xgb_model_optimal = xgb_model_random.best_estimator_.fit(x_train_scale,y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(xgb_model_optimal, x_train_scale, y_train, cv=10,scoring='neg_mean_absolute_error')
np.mean(scores)

-142.29154379729547

In [ ]:
xgb_pred = xgb_model_optimal.predict(x_test_scale)
xgb_pred

array([ 615.17664 , 1493.4008  ,  544.609   ,  585.15356 , 1287.3171  ,
       1894.3147  ,  959.5459  ,  626.56586 ,  317.4308  ,  320.1607  ,
        460.65826 ,  205.06133 ,  349.01074 ,  237.31963 ,  194.60521 ,
        304.99814 ,  495.18777 ,  367.24252 ,  117.05629 ,  776.96216 ,
        235.83563 ,  564.1543  ,  586.378   ,  528.6596  ,  341.18646 ,
        152.8086  ,   94.52139 ,  633.39435 ,  478.46375 ,  560.7769  ,
        959.05286 ,  115.19775 ,  421.8927  ,  264.2224  ,  107.37325 ,
        252.6309  ,  399.7093  ,  692.45337 ,  932.252   ,  355.15155 ,
        537.0888  ,  537.1329  ,  542.89734 ,  616.12115 ,  850.3508  ,
       1280.8694  ,  462.85733 ,  602.99115 ,  387.54645 ,  384.13617 ,
        841.4293  ,  314.35617 , 1227.6877  ,  727.9078  ,  627.4933  ,
        191.9547  ,  763.2428  ,  271.58316 ,  526.07214 ,  103.99247 ,
        381.20886 ,  639.15375 ,  922.35706 ,  473.44946 ,  110.68848 ,
        293.16623 ,  579.8305  , 1332.8624  ,  683.31494 ,  622.

In [ ]:
submission['num'] = xgb_pred
submission.to_csv('xgb_1.csv', index=False)